Analyzing Crime Statistics for the Summer of 2014 in Seattle
============================================================

This is an exploration of (a reduced set of) public crime data in the city of Seattle during the Summer of 2014. The goal is to identify and visualize interesting patterns in this data.

The Data
--------

The data comes in the form of a CSV file, so let's load it up in [Pandas](http://pandas.pydata.org/pandas-docs/stable/) and see what we've got.

In [ ]:
import pandas as pd

In [ ]:
seattle_data = pd.read_csv('data/seattle_incidents_summer_2014.csv')

Well, the first I noticed about it is that type inference is failing on column 9. Let's see what it's supposed to be.

In [ ]:
seattle_data.columns

Apparently, that's `Occurred Date Range End`. Let's take a peak and see what's in that column.

In [ ]:
seattle_data['Occurred Date Range End']

OK, it seems like the problem is some `NaN` entries. Luckily, Pandas let's us specify different strings that mean N/A, so we'll read the file again and tell it that `NaN` is one such value.

While we're at it, let's tell Pandas to parse all these different timestamps. We'll leave it to Pandas to figure out the format, if possible.

In [ ]:
seattle_data = pd.read_csv(
    'data/seattle_incidents_summer_2014.csv',  # The data file.
    na_values=['NaN'],  # 'NaN' means it's a missing value.
    parse_dates=[7,8,9],  # Columns 7-9 contain timestamps; try to parse them as such.
    infer_datetime_format=True  # I don't want to comb over all that data to figure out a format; let Pandas try.
)

# And now set up our ``DataFrame`` in dplython to make it easier to manipulate.
from dplython import mutate, select, DplyFrame, X
dply_seattle = DplyFrame(seattle_data)

# Tell Jupyter we want to render these plots inline in the notebook.
import matplotlib.pyplot as plt
%matplotlib inline
# XKCD-style
# plt.xkcd()
plt.style.use("ggplot")

No warnings or errors this time, so I suspect our changes worked. Let's go ahead and peek at the data to see how it looks.

In [ ]:
seattle_data

That seems reasonable, so let's move on to the next step: visualizing the data and identifying patterns.

Analysis
--------

Our dataset, when augmented with some external datasets, can answer a wide range of questions. Let's choose a few for this exercise.

### Questions to Answer

1. How do incidents vary by time of day?
    1. Which incidents are most common in the evening?
    1. During what periods of the day are robberies most common?

1. How do incidents vary month to month in the Summer 2014 dataset?

1. Which incident types tend to correlate with each other on a day-by-day basis?

1. Do certain crimes correlate with environmental factors such as temperature? NOTE: This question cannot be answered without incorporating additional data. I have reached out to a potential source of weather data, but have not yet heard back from them.

### Question 1: How does the overall incident rate vary by time of day?

### Question 2: How do incidents vary month to month in the Summer 2014 dataset.

To answer this question, we'll start by plotting the total number of incidents by date over the full dataset. The first issue we see is the `Occurred Date or Date Range Start` column is at the wrong level of granularity to answer this question: it is actually a timestamp. For our purposes, we're going to want to roll up by date, so let's add a pair of new columns: `date_of_incident` (the date we'll use to group offenses), and `identity`, which is just a bunch of 1s that we'll be summing later to get total counts for each offense on each day.

In [ ]:
import datetime
def get_date(ts):
    return ts.date()

with_date = (dply_seattle
    >> mutate(
        date_of_incident = X['Occurred Date or Date Range Start'].apply(get_date),
        identity = 1
    )
)
with_date.head()

Now we have the basic info we need for plotting the data as a date-series. Rather than a raw "incident" count by date, I think it would be useful to see the different types of offenses broken out separately. A line chart would be a fairly obvious choice here, but I'd also like to get an idea of the overall number of offenses per day, so let's use a [stacked area plot](http://pandas.pydata.org/pandas-docs/stable/visualization.html#visualization-area-plot) instead, which is happily baked right into the `DataFrame`.

This visualization expects to find the different categories to plot as columns, so we'll need to reshape our data. Once again, the `DataFrame` has what we need: the [`pivot_table`](http://pandas.pydata.org/pandas-docs/stable/reshaping.html#pivot-tables) function.

In [ ]:
date_and_offense_counts = with_date[["date_of_incident", "Offense Type", "identity"]].pivot_table(
    index="date_of_incident",  # We're going to use the date of the offense as the index.
    columns="Offense Type",  # For each type of offense, we'll create a new column.
    values="identity",  # The value we're counting; in the raw data, this is a 1 for
                        # each offense.
    aggfunc=sum,  # We want to roll up our count of `identity` for each category, so
                  # we use the ``sum`` aggregation. If we omit this, the entire pivot
                  # operation will fail because we will have more than one row for
                  # each date.
    dropna=False,  # We don't want to drop missing values; we'll use a "filler" value instead.
    fill_value=0  # 0, to be precise.
)
date_and_offense_counts.head()

Hmm… that's a **lot** of columns. I think we're going to need to pare down our categories a bit. This is going to be a *very* rough cut at this, because doing a good job would likely require a detailed analysis of the coding system used to build the source data, and I don't have that information. 

From a cursory glance at the data, I can see that there are multiple types of offense for assaults, weapons charges, etc., so let's try to consolidate similar types of offense. Each of those "super categories" appears to be broken down using the following scheme: TOP-SUB1-SUB2, so we can get the top-level categories by doing a simple split.

In [ ]:
def extract_top_level_type(x):
    return x.split('-')[0]

consolidated_with_date = (with_date
    >> mutate(
        consolidated_offense = X['Offense Type'].apply(extract_top_level_type)
    )
    >> select(X.date_of_incident, X.consolidated_offense, X.identity)
)

OK, let's see how many categories we're looking at after the consolidation.

In [ ]:
consolidated_with_date.consolidated_offense.nunique()

51 is still too many, so let's try something a bit more… extensive.

According to [this LegalMatch article](http://www.legalmatch.com/law-library/article/what-are-the-different-types-of-crimes.html), crimes are generally divided into the following categories:

<dl>
  <dt>Personal Crimes</dt><dd>crimes that result in physical or mental harm to another person, e.g., assault</dd>
  <dt>Property Crimes</dt><dd>crimes that do not necessarily involve harm to another person, instead involving an interference with another person’s right to use or enjoy their property, e.g., burglary</dd>
  <dt>Inchoate Crimes</dt><dd>crimes that were begun, but not completed, e.g., solicitation. This requires that a person take a substantial step to complete a crime, as opposed to just “intend” to commit a crime.</dd>
  <dt>Statutory Crimes</dt><dd>a violation of a specific state or federal statute and can involve either property offenses or personal offense, e.g., DUI.</dd>
</dl>

I have manually mapped each of the `Offense Type` entries back to these 4 categories, although based on my rudimentary undertanding, none of them actually qualified as "Inchoate", and some would have fit under either "Personal" or "Property", e.g, all of the `ROBBERY-…` entries.

In [ ]:
_personal = set([
    'ASSLT-AGG-BODYFORCE',
    'ASSLT-AGG-GUN',
    'ASSLT-AGG-POLICE-BODYFORCE',
    'ASSLT-AGG-POLICE-GUN',
    'ASSLT-AGG-POLICE-WEAPON',
    'ASSLT-AGG-WEAPON',
    'ASSLT-NONAGG',
    'ASSLT-NONAGG-POLICE',
    'DISPUTE-OTH',
    'ENDANGERMENT',
    'HARASSMENT',
    'HOMICIDE-JUST-GUN',
    'HOMICIDE-JUST-WEAPON',
    'HOMICIDE-PREMEDITATED-GUN',
    'INJURY - ACCIDENTAL',
    'INJURY - OTHER',
    'MALICIOUS HARASSMENT',
    'ROBBERY-BANK-BODYFORCE',
    'ROBBERY-BANK-GUN',
    'ROBBERY-BANK-WEAPON',
    'ROBBERY-BUSINESS-BODYFORCE',
    'ROBBERY-BUSINESS-GUN',
    'ROBBERY-BUSINESS-WEAPON',
    'ROBBERY-RESIDENCE-BODYFORCE',
    'ROBBERY-RESIDENCE-GUN',
    'ROBBERY-RESIDENCE-WEAPON',
    'ROBBERY-STREET-BODYFORCE',
    'ROBBERY-STREET-GUN',
    'ROBBERY-STREET-WEAPON',
    'THREATS-KILL',
    'THREATS-OTHER',
    'THREATS-WEAPON',
])
_property = set([
    'BURGLARY-FORCE-NONRES',
    'BURGLARY-FORCE-RES',
    'BURGLARY-NOFORCE-NONRES',
    'BURGLARY-NOFORCE-RES',
    'BURGLARY-SECURE PARKING-NONRES',
    'BURGLARY-SECURE PARKING-RES',
    'COUNTERFEIT',
    'DISPUTE-CIVIL PROPERTY (AUTO)',
    'DISPUTE-CIVIL PROPERTY (NON AU',
    'EMBEZZLE',
    'FORGERY-CHECK',
    'FORGERY-CREDIT CARD',
    'FORGERY-OTH',
    'FRAUD-CHECK',
    'FRAUD-COMPUTER',
    'FRAUD-CREDIT CARD',
    'FRAUD-IDENTITY THEFT',
    'FRAUD-OTHER',
    'FRAUD-WIRE-ELECTRONIC',
    'PROP RECOVERED-OTHER AGENCY',
    'PROPERTY DAMAGE - GRAFFITI',
    'PROPERTY DAMAGE-NON RESIDENTIA',
    'PROPERTY DAMAGE-RESIDENTIAL',
    'PROPERTY FOUND',
    'PROPERTY LOST',
    'PROPERTY LOST - POLICE EQUIPME',
    'PROPERTY STOLEN-POSSESS',
    'PROPERTY STOLEN-SELL',
    'PROPERTY STOLEN-TRAFFICKING',
    'RECKLESS BURNING',
    'THEFT OF SERVICES',
    'THEFT-AUTO PARTS',
    'THEFT-AUTOACC',
    'THEFT-BICYCLE',
    'THEFT-BOAT',
    'THEFT-BUILDING',
    'THEFT-CARPROWL',
    'THEFT-COINOP',
    'THEFT-LICENSE PLATE',
    'THEFT-MAIL',
    'THEFT-OTH',
    'THEFT-PKPOCKET',
    'THEFT-PRSNATCH',
    'THEFT-SHOPLIFT',
    'TRESPASS',
    'VEH-RCVD-FOR OTHER AGENCY',
    'VEH-THEFT-AUTO',
    'VEH-THEFT-MTRCYCLE',
    'VEH-THEFT-OTHVEH',
    'VEH-THEFT-TRAILER',
    'VEH-THEFT-TRUCK',
])
_inchoate = set()
_statutory = set([
    'ANIMAL-BITE',
    'ANIMAL-CRUELTY',
    'ANIMAL-OTH',
    'BIAS INCIDENT',
    'DISORDERLY CONDUCT',
    'DISTURBANCE-NOISE',
    'DISTURBANCE-OTH',
    'DRIVE-BY',
    'DUI-DRUGS',
    'DUI-LIQUOR',
    'ELUDING-FELONY FLIGHT',
    'ESCAPE',
    'FALSE REPORT',
    'FIREWORK-POSSESS',
    'FIREWORK-USE',
    'ILLEGAL DUMPING',
    'LIQUOR LAW VIOLATION',
    'NARC-DRUG TRAFFIC LOITERING',
    'NARC-EQUIPMENT/PARAPHENALIA',
    'NARC-FORGERY-PRESCRIPTION',
    'NARC-FOUND-AMPHETAMINE',
    'NARC-FOUND-COCAINE',
    'NARC-FOUND-HEROIN',
    'NARC-FOUND-MARIJU',
    'NARC-FOUND-METH',
    'NARC-FOUND-OPIUM',
    'NARC-FOUND-OTHER',
    'NARC-FOUND-SYNTHETIC',
    'NARC-FRAUD-PRESCRIPTION',
    'NARC-POSSESS-AMPHETAMINE',
    'NARC-POSSESS-COCAINE',
    'NARC-POSSESS-HALLUCINOGEN',
    'NARC-POSSESS-HEROIN',
    'NARC-POSSESS-MARIJU',
    'NARC-POSSESS-METH',
    'NARC-POSSESS-OTHER',
    'NARC-POSSESS-PILL/TABLET',
    'NARC-PRODUCE-MARIJU',
    'NARC-SELL-AMPHETAMINE',
    'NARC-SELL-COCAINE',
    'NARC-SELL-HEROIN',
    'NARC-SELL-MARIJU',
    'NARC-SELL-METH',
    'NARC-SELL-SYNTHETIC',
    'NARC-SMUGGLE-OTHER',
    'OBSTRUCT',
    'PORNOGRAPHY-OBSCENE MATERIAL',
    'PROSTITUTION',
    'PROSTITUTION LOITERING',
    'PROSTITUTION PATRONIZING',
    'PROSTITUTION-ASSIST-PROMOTE',
    'TRAFFIC',
    'URINATING/DEFECATING-IN PUBLIC',
    'VIOL-COURT ORDER',
    'WARRANT-FUGITIVE',
    'WARRARR-FELONY',
    'WARRARR-MISDEMEANOR',
    'WEAPON-CONCEALED',
    'WEAPON-DISCHARGE',
    'WEAPON-POSSESSION',
    'WEAPON-SELLING',
    'WEAPON-SURRENDER-EXCLUDING FIR',
    'WEAPON-UNLAWFUL USE',
    '[INC - CASE DC USE ONLY]'
])

def map_offense_type_to_category(ot):
    if ot in _personal:
        return "PERSONAL"
    if ot in _property:
        return "PROPERTY"
    if ot in _inchoate:
        return "INCHOATE"
    if ot in _statutory:
        return "STATUTORY"
    return None

consolidated_with_date = (with_date
    >> mutate(
        consolidated_offense = X['Offense Type'].apply(map_offense_type_to_category)
    )
    >> select(X.date_of_incident, X.consolidated_offense, X.identity)
)

OK, let's see how many categories we're looking at after *this* consolidation (it should be 3).

In [ ]:
consolidated_with_date.consolidated_offense.nunique()

3 is the answer we were looking for, so let's do an area plot now.

In [ ]:
date_and_offense_counts = consolidated_with_date.pivot_table(
    index="date_of_incident",  # We're going to use the date of the offense as the index.
    columns="consolidated_offense",  # For each type of offense, we'll create a new column.
    values="identity",  # The value we're counting; in the raw data, this is a 1 for
                        # each offense.
    aggfunc=sum,  # We want to roll up our count of `identity` for each category, so
                  # we use the ``sum`` aggregation. If we omit this, the entire pivot
                  # operation will fail because we will have more than one row for
                  # each date.
    dropna=False,  # We don't want to drop missing values; we'll use a "filler" value instead.
    fill_value=0  # 0, to be precise.
)
date_and_offense_counts.plot.area(figsize=(12,8))

We can see that the overall crime rate begins to drop off in August, driven primarily by a drop in property crime rates. Rates of personal and statutory crimes do not seem to be affected as deeply by the month.